In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
from pathlib import Path

# Configurable parts
SOURCE = "dukascopy"
SYMBOL = "usdjpy"
MINUTES = 1
EVENT = '58m-dollar'
START_DATE = "2020-01-01"
END_DATE = "2024-12-31"

# Build base name
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{EVENT}-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
NORMALIZED_DIR = BASE_DIR / "normalized"
SCALER_DIR = BASE_DIR / "scalers"

# Final paths
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_processed.pkl"
NORMALIZED_FILE_PATH = NORMALIZED_DIR / f"{RESAMPLED_NAME}_normalized.pkl"
STD_SCALER_PATH = SCALER_DIR / f"{RESAMPLED_NAME}_standard_scaler.pkl"
MINMAX_SCALER_PATH = SCALER_DIR / f"{RESAMPLED_NAME}_minmax_scaler.pkl"

In [3]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
df.head()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-02 03:35:29.827,108.6945,108.6995,108.6905,108.6955,533610.002041,0.002335,13.187422,0.001,0.000009,0.000009,...,0.707107,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025
2020-01-02 03:47:19.816,108.6960,108.7135,108.6895,108.6905,533940.002441,0.002247,13.188041,-0.005,-0.000046,-0.000046,...,0.707107,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025
2020-01-02 04:01:46.852,108.6900,108.7115,108.6870,108.7005,537560.003281,0.002303,13.194798,0.010,0.000092,0.000092,...,0.500000,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025
2020-01-02 04:17:30.077,108.7000,108.7145,108.6990,108.7105,533880.000949,0.002420,13.187928,0.010,0.000092,0.000092,...,0.500000,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025
2020-01-02 04:35:15.235,108.7100,108.7300,108.7080,108.7255,535220.003963,0.002204,13.190435,0.015,0.000138,0.000138,...,0.500000,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025


In [4]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'log_volume',
       'close_delta', 'close_return', 'close_log_return', 'ret_mean_5',
       'ret_mean_10', 'ema5', 'ema5_slope', 'ema20', 'ema20_slope', 'atr14',
       'atr20', 'vol_adj_return', 'close_to_atr', 'bb_upper', 'bb_lower',
       'bb_mavg', 'bb_width', 'bb_position', 'donchian_upper',
       'donchian_lower', 'donchian_mid', 'donchian_width', 'stoch_k',
       'stoch_d', 'rsi14', 'macd', 'macd_signal', 'macd_diff', 'unix_time',
       'hour', 'hour_sin', 'hour_cos', 'dow', 'dow_sin', 'dow_cos', 'dom',
       'dom_sin', 'dom_cos', 'month', 'month_sin', 'month_cos'],
      dtype='object')

In [5]:
df.describe()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
count,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,...,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06
mean,1.375142e+02,1.375263e+02,1.375021e+02,1.375142e+02,4.313956e+05,6.826283e-03,1.296698e+01,2.650954e-05,2.141538e-07,2.017857e-07,...,-2.184476e-01,2.084740e+00,3.556377e-01,-1.062608e-01,1.566359e+01,1.597130e-02,-2.044634e-02,6.930031e+00,-5.399113e-02,8.130608e-02
std,1.495205e+01,1.495112e+01,1.495295e+01,1.495204e+01,6.117032e+04,4.653172e-03,1.199805e-01,2.114303e-02,1.572735e-04,1.572799e-04,...,6.960557e-01,1.429296e+00,5.227438e-01,7.674437e-01,8.801866e+00,7.098904e-01,7.038346e-01,3.546064e+00,7.076949e-01,6.997447e-01
min,1.012020e+02,1.012975e+02,1.011820e+02,1.012045e+02,3.582400e+05,1.000000e-03,1.278896e+01,-1.447000e+00,-1.247610e-02,-1.255458e-02,...,-1.000000e+00,0.000000e+00,-7.818315e-01,-9.009689e-01,1.000000e+00,-9.987165e-01,-9.948693e-01,1.000000e+00,-1.000000e+00,-1.000000e+00
25%,1.320170e+02,1.320310e+02,1.320040e+02,1.320170e+02,3.941400e+05,5.209159e-03,1.288446e+01,-1.000000e-02,-7.295967e-05,-7.296233e-05,...,-8.660254e-01,1.000000e+00,0.000000e+00,-9.009689e-01,8.000000e+00,-7.247928e-01,-7.587581e-01,4.000000e+00,-8.660254e-01,-5.000000e-01
50%,1.415755e+02,1.415885e+02,1.415630e+02,1.415760e+02,4.133200e+05,6.345238e-03,1.293198e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,-5.000000e-01,2.000000e+00,4.338837e-01,-2.225209e-01,1.500000e+01,1.011683e-01,-5.064917e-02,7.000000e+00,-2.449294e-16,6.123234e-17
75%,1.483115e+02,1.483225e+02,1.483014e+02,1.483115e+02,4.427900e+05,7.704918e-03,1.300085e+01,1.050000e-02,7.428707e-05,7.428431e-05,...,2.588190e-01,3.000000e+00,7.818315e-01,6.234898e-01,2.300000e+01,7.247928e-01,6.889669e-01,1.000000e+01,5.000000e-01,8.660254e-01
max,1.619510e+02,1.619510e+02,1.619440e+02,1.619505e+02,5.001500e+06,2.646471e-01,1.542525e+01,1.616000e+00,1.139348e-02,1.132906e-02,...,1.000000e+00,6.000000e+00,9.749279e-01,1.000000e+00,3.100000e+01,9.987165e-01,1.000000e+00,1.200000e+01,1.000000e+00,1.000000e+00


In [6]:
FEATURES_COLS = [
    # Basic Data
    'close_log_return',
    'log_volume',
    'spread',

    # Other
    'ret_mean_5',
    'ret_mean_10'
]
TIME_FEATURE_COLS = [
    'hour_cos',
    'dow_cos',
    'dom_cos',
    'month_cos',
]
TA_MIN_MAX = [
    # TA
    'rsi14',
]
TA_STD = [
    "ema5",
    "ema5_slope",
    "ema20",
    "ema20_slope",
    "atr14",
    "atr20",
    "vol_adj_return",
    "close_to_atr",
    "bb_upper",
    "bb_lower",
    "bb_mavg",
    "bb_width",
    "bb_position",
    "donchian_upper",
    "donchian_lower",
    "donchian_mid",
    "donchian_width",
    "stoch_k",
    "stoch_d",
    "macd",
    "macd_signal",
    "macd_diff",
]

In [7]:
COLS_TO_STD = FEATURES_COLS + TA_STD
COLS_TO_MIN_MAX = TA_MIN_MAX

In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

In [9]:
df[COLS_TO_STD] = standard_scaler.fit_transform(df[COLS_TO_STD])
df[COLS_TO_MIN_MAX] = minmax_scaler.fit_transform(df[COLS_TO_MIN_MAX])

In [10]:
df.describe()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
count,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,...,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06,1.822438e+06
mean,1.375142e+02,1.375263e+02,1.375021e+02,1.375142e+02,4.313956e+05,7.585619e-17,1.073165e-14,2.650954e-05,2.141538e-07,-2.086669e-17,...,-2.184476e-01,2.084740e+00,3.556377e-01,-1.062608e-01,1.566359e+01,1.597130e-02,-2.044634e-02,6.930031e+00,-5.399113e-02,8.130608e-02
std,1.495205e+01,1.495112e+01,1.495295e+01,1.495204e+01,6.117032e+04,1.000000e+00,1.000000e+00,2.114303e-02,1.572735e-04,1.000000e+00,...,6.960557e-01,1.429296e+00,5.227438e-01,7.674437e-01,8.801866e+00,7.098904e-01,7.038346e-01,3.546064e+00,7.076949e-01,6.997447e-01
min,1.012020e+02,1.012975e+02,1.011820e+02,1.012045e+02,3.582400e+05,-1.252110e+00,-1.483700e+00,-1.447000e+00,-1.247610e-02,-7.982450e+01,...,-1.000000e+00,0.000000e+00,-7.818315e-01,-9.009689e-01,1.000000e+00,-9.987165e-01,-9.948693e-01,1.000000e+00,-1.000000e+00,-1.000000e+00
25%,1.320170e+02,1.320310e+02,1.320040e+02,1.320170e+02,3.941400e+05,-3.475317e-01,-6.877140e-01,-1.000000e-02,-7.295967e-05,-4.651844e-01,...,-8.660254e-01,1.000000e+00,0.000000e+00,-9.009689e-01,8.000000e+00,-7.247928e-01,-7.587581e-01,4.000000e+00,-8.660254e-01,-5.000000e-01
50%,1.415755e+02,1.415885e+02,1.415630e+02,1.415760e+02,4.133200e+05,-1.033800e-01,-2.916844e-01,0.000000e+00,0.000000e+00,-1.282972e-03,...,-5.000000e-01,2.000000e+00,4.338837e-01,-2.225209e-01,1.500000e+01,1.011683e-01,-5.064917e-02,7.000000e+00,-2.449294e-16,6.123234e-17
75%,1.483115e+02,1.483225e+02,1.483014e+02,1.483115e+02,4.427900e+05,1.888250e-01,2.823533e-01,1.050000e-02,7.428707e-05,4.710237e-01,...,2.588190e-01,3.000000e+00,7.818315e-01,6.234898e-01,2.300000e+01,7.247928e-01,6.889669e-01,1.000000e+01,5.000000e-01,8.660254e-01
max,1.619510e+02,1.619510e+02,1.619440e+02,1.619505e+02,5.001500e+06,5.540755e+01,2.048893e+01,1.616000e+00,1.139348e-02,7.202998e+01,...,1.000000e+00,6.000000e+00,9.749279e-01,1.000000e+00,3.100000e+01,9.987165e-01,1.000000e+00,1.200000e+01,1.000000e+00,1.000000e+00


In [11]:
print(df[COLS_TO_STD].mean())         # Should be ~0
print(df[COLS_TO_STD].std(ddof=0))

close_log_return   -2.086669e-17
log_volume          1.073165e-14
spread              7.585619e-17
ret_mean_5          5.489592e-18
ret_mean_10         2.308124e-18
ema5                3.193945e-16
ema5_slope         -5.052920e-18
ema20              -7.665467e-16
ema20_slope         1.247635e-18
atr14              -1.357426e-16
atr20              -3.193945e-16
vol_adj_return     -1.241396e-17
close_to_atr        2.800940e-17
bb_upper           -2.555156e-16
bb_lower            1.277578e-16
bb_mavg             8.304256e-16
bb_width           -8.384105e-17
bb_position         1.871452e-17
donchian_upper      1.277578e-16
donchian_lower     -1.916367e-16
donchian_mid        6.387889e-17
donchian_width     -5.589403e-17
stoch_k            -1.462228e-16
stoch_d            -2.106007e-16
macd               -3.742904e-18
macd_signal         3.867667e-18
macd_diff           1.182719e-17
dtype: float64
close_log_return    1.0
log_volume          1.0
spread              1.0
ret_mean_5          1.

In [12]:
import joblib
joblib.dump(standard_scaler, STD_SCALER_PATH)
joblib.dump(minmax_scaler, MINMAX_SCALER_PATH)
df.to_pickle(NORMALIZED_FILE_PATH)